In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Dash components
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
JupyterDash.infer_jupyter_proxy_config()

# Data / plotting
import pandas as pd
import plotly.express as px
import base64
import os

# ---- CRUD module ----
from CRUD_Python_Module import AnimalShelter


###########################
# Data Manipulation / Model
###########################
USERNAME = "acuser"                 # <-- keep your username
PASSWORD = "PUT_A_STRONG_PASSWORD_HERE"  # <-- put YOUR real password here
HOST = "localhost"
PORT = 27017

db = AnimalShelter(USERNAME, PASSWORD, host=HOST, port=PORT)

# initial dataset (unfiltered)
docs = db.read({})
df = pd.DataFrame.from_records(docs) if docs else pd.DataFrame()

# drop _id if present
if "_id" in df.columns:
    df = df.drop(columns=["_id"])

# convenience: column name fallbacks (older CSVs sometimes differ)
LAT_COL = "location_lat" if "location_lat" in df.columns else (df.columns[13] if len(df.columns) > 13 else None)
LON_COL = "location_long" if "location_long" in df.columns else (df.columns[14] if len(df.columns) > 14 else None)
BREED_COL = "breed" if "breed" in df.columns else (df.columns[4] if len(df.columns) > 4 else None)
NAME_COL  = "name"  if "name"  in df.columns else (df.columns[9] if len(df.columns) > 9 else None)

def query_for(option: str) -> dict:
    """Return a MongoDB query matching the rescue filter."""
    if option == "water":
        return {
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Male",
        }
    if option == "mountain":
        return {
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog",
                              "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
        }
    if option == "disaster":
        return {
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever",
                              "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300},
        }
    return {}  # reset / default


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Prefer using assets/ for the logo (Dash serves automatically). If not present, fall back to base64.
logo_src = None
assets_logo_path = os.path.join("assets", "grazioso_logo.png")
if os.path.exists(assets_logo_path):
    logo_src = "assets/grazioso_logo.png"
else:
    fallback = "Grazioso Salvare Logo.png"  # your file in code_files/
    if os.path.exists(fallback):
        encoded = base64.b64encode(open(fallback, "rb").read()).decode()
        logo_src = f"data:image/png;base64,{encoded}"

controls = html.Div(
    [
        html.Hr(),
        html.Label("Rescue Type Filter:"),
        dcc.RadioItems(
            id="filter-type",
            options=[
                {"label": "Water Rescue", "value": "water"},
                {"label": "Mountain/Wilderness Rescue", "value": "mountain"},
                {"label": "Disaster/Individual Tracking", "value": "disaster"},
                {"label": "Reset (All)", "value": "reset"},
            ],
            value="reset",
            inline=True,
        ),
        html.Hr(),
    ]
)

app.layout = html.Div(
    [
        html.Center(
            [
                html.B(html.H1("CS-340 Dashboard — Zachary Locke")),
                html.Div(
                    html.Img(src=logo_src, style={"height": "70px"}) if logo_src else html.Div(),
                    style={"marginTop": "4px"},
                ),
            ]
        ),
        controls,
        dash_table.DataTable(
            id="datatable-id",
            columns=[{"name": c, "id": c, "deletable": False, "selectable": True} for c in df.columns],
            data=df.to_dict("records"),
            # friendly features
            filter_action="native",
            sort_action="native",
            sort_mode="multi",
            page_action="native",
            page_size=10,
            row_selectable="single",
            selected_rows=[0],
            style_table={"overflowX": "auto"},
            style_cell={"textAlign": "left", "padding": "6px", "minWidth": 100},
            style_header={"fontWeight": "bold"},
        ),
        html.Br(),
        html.Hr(),
        html.Div(
            className="row",
            style={"display": "flex"},
            children=[
                html.Div(id="graph-id", className="col s12 m6"),
                html.Div(id="map-id", className="col s12 m6"),
            ],
        ),
    ]
)


#############################################
# Interaction Between Components / Controller
#############################################

# Update table based on radio filter (Mongo query)
@app.callback(
    Output("datatable-id", "data"),
    Input("filter-type", "value"),
)
def update_dashboard(filter_type):
    q = query_for(filter_type)
    docs = db.read(q) if q else db.read({})
    dff = pd.DataFrame.from_records(docs) if docs else pd.DataFrame(columns=df.columns)
    if "_id" in dff.columns:
        dff = dff.drop(columns=["_id"])
    return dff.to_dict("records")


# Pie chart that reflects current table view
@app.callback(
    Output("graph-id", "children"),
    Input("datatable-id", "derived_virtual_data"),
)
def update_graphs(viewData):
    dff = pd.DataFrame(viewData) if viewData else pd.DataFrame()
    if dff.empty:
        fig = px.pie(values=[1], names=["No data"], title="Distribution")
    else:
        # Prefer outcome_type; fallback to breed
        col = "outcome_type" if "outcome_type" in dff.columns else (BREED_COL or dff.columns[0])
        fig = px.pie(dff, names=col, title=f"Distribution by {col.replace('_',' ').title()}")
    return dcc.Graph(figure=fig)


# Highlight selected columns
@app.callback(
    Output("datatable-id", "style_data_conditional"),
    Input("datatable-id", "selected_columns"),
)
def update_styles(selected_columns):
    selected_columns = selected_columns or []
    return [{"if": {"column_id": c}, "background_color": "#D2F3FF"} for c in selected_columns]


# Update Leaflet map from selected row in the current table view
@app.callback(
    Output("map-id", "children"),
    [Input("datatable-id", "derived_virtual_data"),
     Input("datatable-id", "derived_virtual_selected_rows")],
)
def update_map(viewData, index):
    dff = pd.DataFrame(viewData) if viewData else pd.DataFrame()
    if dff.empty or LAT_COL is None or LON_COL is None:
        # Empty or missing geocoords—return a centered base map
        return [
            dl.Map(
                style={"width": "1000px", "height": "500px"},
                center=[30.75, -97.48],
                zoom=9,
                children=[dl.TileLayer(id="base-layer-id")],
            )
        ]

    row = index[0] if index else 0
    row = max(0, min(row, len(dff) - 1))  # guard bounds

    try:
        lat = float(dff.iloc[row][LAT_COL])
        lon = float(dff.iloc[row][LON_COL])
    except Exception:
        lat, lon = 30.75, -97.48

    breed = str(dff.iloc[row][BREED_COL]) if (BREED_COL and BREED_COL in dff.columns) else "Unknown"
    name  = str(dff.iloc[row][NAME_COL])  if (NAME_COL  and NAME_COL  in dff.columns) else "(unknown)"

    return [
        dl.Map(
            style={"width": "1000px", "height": "500px"},
            center=[lat, lon],
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[lat, lon],
                    children=[dl.Tooltip(breed), dl.Popup([html.H1("Animal Name"), html.P(name)])],
                ),
            ],
        )
    ]


# Run the app 
app.run_server(mode="inline", port=8051)